In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [3]:
# load the dataset
import platform

if platform.system() == 'Linux':
    ratings = pd.read_csv('/content/drive/MyDrive/deep learning/deep_learning_a-z/Part 5 - Boltzmann Machines (BM)/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
    movies = pd.read_csv('/content/drive/MyDrive/deep learning/deep_learning_a-z/Part 5 - Boltzmann Machines (BM)/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
    users = pd.read_csv('/content/drive/MyDrive/deep learning/deep_learning_a-z/Part 5 - Boltzmann Machines (BM)/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
else:
    ratings = pd.read_csv('./ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
    movies = pd.read_csv('./ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
    users = pd.read_csv('./ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [3]:
# Preparar el conjunto de entrenamiento y el conjunto de testing
training_set = pd.read_csv('./ml-100k/u1.base', sep = "\t", header = None)
training_set = np.array(training_set, dtype = "int")
test_set = pd.read_csv('./ml-100k/u1.test', sep = "\t", header = None)
test_set = np.array(test_set, dtype = "int")

In [8]:
# Preparar el conjunto de entrenamiento y el conjunto de testing
training_set = pd.read_csv('./ml-1m/training_set.csv')
training_set = np.array(training_set, dtype = "int")
test_set = pd.read_csv('./ml-1m/test_set.csv')
test_set = np.array(test_set, dtype = "int")

In [9]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_films = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

In [10]:
# create the function for the input of the pytorch network
def convert(data):
    new_data = []
    for id_user in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_user]
        id_ratings = data[:, 2][data[:, 0] == id_user]
        ratings = np.zeros(nb_films)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)

In [11]:
# convert the data to tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [12]:
# convert the rankings to binary values
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1


In [13]:
# create the architecture of the Bolztmann Machine
class RBM(nn.Module):
    def __init__(self, vn, hn):
        super(RBM, self).__init__()
        self.W = torch.randn(vn, hn)
        self.a = torch.randn(1, hn)
        self.b = torch.randn(1, vn)

    def sample_h(self, x):
        wx = torch.mm(x, self.W) # mini_batch_size x nh
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)

    def sample_v(self, y):
        wy = torch.mm(y, self.W.t()) # mini_batch_size x nv
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)

    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

In [14]:
# define the model
nv = len(training_set[0])
nh = 100
batch_size = 100

rbm = RBM(nv, nh)

In [15]:
# train the restricted Bolztmann machine
nb_epochs = 10
for epoch in range(1, nb_epochs + 1):
    training_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user + batch_size]
        v0 = training_set[id_user:id_user + batch_size]
        ph0, _ = rbm.sample_h(v0)
        for k in range(10):
            _, hk = rbm.sample_h(vk)
            _, vk = rbm.sample_v(hk)
            vk[v0 < 0] = v0[v0 < 0]
        phk, _ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        training_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[vk >= 0]))
        s += 1.
    print(f'Epoch: ' + str(epoch))
    print('Loss: ' + str(training_loss / s))

Epoch: 1
Loss: tensor(0.2415)
Epoch: 2
Loss: tensor(0.2286)
Epoch: 3
Loss: tensor(0.2285)
Epoch: 4
Loss: tensor(0.2277)
Epoch: 5
Loss: tensor(0.2284)
Epoch: 6
Loss: tensor(0.2278)
Epoch: 7
Loss: tensor(0.2285)
Epoch: 8
Loss: tensor(0.2279)
Epoch: 9
Loss: tensor(0.2281)
Epoch: 10
Loss: tensor(0.2278)


In [16]:
# save the restricted Bolztmann machine
PATH = "rbm.pt"
torch.save(rbm.state_dict(), PATH)

In [ ]:
# Load the model
rbm = RBM(nv, nh)
rbm.load_state_dict(torch.load(PATH))

In [17]:
# testing the rbm
testing_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user + 1]
    vt = test_set[id_user:id_user + 1]
    if len(vt[vt>=0]) > 0: # check if it have rated films
        _, h = rbm.sample_h(v)
        _, v = rbm.sample_v(h)
        testing_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0]))
        s += 1.
        print('Testing Loss: ' + str(testing_loss / s))

Testing Loss: tensor(0.)
Testing Loss: tensor(0.1562)
Testing Loss: tensor(0.1811)
Testing Loss: tensor(0.1358)
Testing Loss: tensor(0.1847)
Testing Loss: tensor(0.1724)
Testing Loss: tensor(0.1478)
Testing Loss: tensor(0.1436)
Testing Loss: tensor(0.1405)
Testing Loss: tensor(0.1404)
Testing Loss: tensor(0.1571)
Testing Loss: tensor(0.1579)
Testing Loss: tensor(0.1628)
Testing Loss: tensor(0.1869)
Testing Loss: tensor(0.1931)
Testing Loss: tensor(0.2088)
Testing Loss: tensor(0.2065)
Testing Loss: tensor(0.2104)
Testing Loss: tensor(0.2108)
Testing Loss: tensor(0.2086)
Testing Loss: tensor(0.2146)
Testing Loss: tensor(0.2177)
Testing Loss: tensor(0.2191)
Testing Loss: tensor(0.2149)
Testing Loss: tensor(0.2120)
Testing Loss: tensor(0.2215)
Testing Loss: tensor(0.2175)
Testing Loss: tensor(0.2137)
Testing Loss: tensor(0.2088)
Testing Loss: tensor(0.2019)
Testing Loss: tensor(0.1997)
Testing Loss: tensor(0.2038)
Testing Loss: tensor(0.2032)
Testing Loss: tensor(0.2044)
Testing Loss: tens